In [1]:
# !pip install pymysql sqlalchemy

In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

import getpass  # to get the password without showing the input
password = getpass.getpass()

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)


········


In [3]:
#Create a query or queries to extract the information you think may be relevant for building the prediction model. 
#It should include some film features and some rental features.

In [4]:
query = '''select rental_id,rental_date,film_id,r.inventory_id,release_year,length,name from rental r
join inventory i ON r.inventory_id = i.inventory_id
join film f using (film_id)
join film_category fc using (film_id)
join category c using (category_id)
where r.rental_date between '2005-05-01' and '2005-06-30';'''

data = pd.read_sql_query(query, engine)
data

,rental_id,rental_date,film_id,inventory_id,release_year,length,name
0,1,2005-05-24 22:53:30,80,367,2006,148,Family
1,2,2005-05-24 22:54:33,333,1525,2006,126,Music
2,3,2005-05-24 23:03:39,373,1711,2006,156,Children
3,4,2005-05-24 23:04:41,535,2452,2006,181,Horror
4,5,2005-05-24 23:05:21,450,2079,2006,84,Children
...,...,...,...,...,...,...,...
3462,3465,2005-06-21 22:10:01,326,1488,2006,69,Animation
3463,3466,2005-06-21 22:13:33,81,371,2006,103,Sci-Fi
3464,3467,2005-06-21 22:19:25,159,729,2006,58,Comedy
3465,3468,2005-06-21 22:43:45,637,2899,2006,131,Sci-Fi


In [5]:
data['rental_month'] = data['rental_date'].dt.month
data = data.drop(columns=['rental_date','inventory_id'])
data

,rental_id,film_id,release_year,length,name,rental_month
0,1,80,2006,148,Family,5
1,2,333,2006,126,Music,5
2,3,373,2006,156,Children,5
3,4,535,2006,181,Horror,5
4,5,450,2006,84,Children,5
...,...,...,...,...,...,...
3462,3465,326,2006,69,Animation,6
3463,3466,81,2006,103,Sci-Fi,6
3464,3467,159,2006,58,Comedy,6
3465,3468,637,2006,131,Sci-Fi,6


#Preparation

In [6]:
data.shape

(3467, 6)

In [7]:
data.dtypes

rental_id        int64
film_id          int64
release_year     int64
length           int64
name            object
rental_month     int32
dtype: object

In [8]:
data.isna().sum()

rental_id       0
film_id         0
release_year    0
length          0
name            0
rental_month    0
dtype: int64

In [9]:
current_month = 6 #June
previous_month = 5 #May

# Create a new column `rented_last_month`
data['rented_last_month'] = data['rental_month'] == previous_month

# Group by film_id and check if it was rented in the previous month
rented_last_month_data = data.groupby('film_id')['rented_last_month'].any().reset_index()


# Merge this result with the original DataFrame to show film info and rented_last_month flag
new_data = data.drop_duplicates().merge(rented_last_month_data, on='film_id')

#Removing the additional colonne that was created which was only looking at the month and not the film_id.
new_data = new_data.drop(columns=['rented_last_month_x'])

new_data


,rental_id,film_id,release_year,length,name,rental_month,rented_last_month_y
0,1,80,2006,148,Family,5,True
1,1577,80,2006,148,Family,6,True
2,2,333,2006,126,Music,5,True
3,769,333,2006,126,Music,5,True
4,1449,333,2006,126,Music,6,True
...,...,...,...,...,...,...,...
3462,3337,261,2006,171,Documentary,6,False
3463,3346,960,2006,128,Documentary,6,False
3464,3358,384,2006,49,Music,6,False
3465,3436,601,2006,79,Travel,6,False


In [10]:
new_data['rented_last_month_y'].value_counts()


rented_last_month_y
True     3082
False     385
Name: count, dtype: int64

#This data is imbalanced which means that it is highly likely that our model will not be predicting based on the features but will be guessing the results as True since most movies are rented 2 months in a row.

Data Processing

In [11]:
y = new_data['rented_last_month_y']
X = new_data.drop(['rented_last_month_y', 'rental_id'], axis=1)

In [12]:
X = pd.get_dummies(X, columns=['name'])
X

,film_id,release_year,length,rental_month,name_Action,name_Animation,name_Children,name_Classics,name_Comedy,name_Documentary,name_Drama,name_Family,name_Foreign,name_Games,name_Horror,name_Music,name_New,name_Sci-Fi,name_Sports,name_Travel
0,80,2006,148,5,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,80,2006,148,6,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,333,2006,126,5,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,333,2006,126,5,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
4,333,2006,126,6,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3462,261,2006,171,6,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3463,960,2006,128,6,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3464,384,2006,49,6,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3465,601,2006,79,6,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


#Model Benchmark

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)

predictions = classification.predict(X_test)
print(confusion_matrix(y_test, predictions))

[[  0  77]
 [  0 617]]


In [14]:
# get the accuracy:
classification.score(X_test, y_test)

0.8890489913544669

In [15]:
predictions = classification.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00        77
        True       0.89      1.00      0.94       617

    accuracy                           0.89       694
   macro avg       0.44      0.50      0.47       694
weighted avg       0.79      0.89      0.84       694



C:\Users\kaohs\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kaohs\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kaohs\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
# to get the probability instead
predictions_ = classification.predict_proba(X_test)
predictions_

array([[0.14224479, 0.85775521],
       [0.25395953, 0.74604047],
       [0.09100154, 0.90899846],
       ...,
       [0.24614655, 0.75385345],
       [0.11404922, 0.88595078],
       [0.09262718, 0.90737282]])

The imbalanced data significantly affects the performance of the prediction model. At first glance, the high accuracy might suggest the model is performing well. However, a closer look at the classification report reveals that the model is unable to correctly predict films that were "not rented last month." This issue arises because the data is heavily skewed towards the "rented last month" class (True). In the test set, there are 617 instances of "True" and only 77 instances of "False," making it difficult for the model to accurately predict the minority class ("False").